In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('US_Accidents_Dec20.csv')

In [ ]:
dataset.head()

In [ ]:
data_set = dataset.iloc[:500, [3,23,24,25,26,27,29]]
data_set.head()

In [ ]:
data_set.head()

In [ ]:
X = data_set.iloc[:,1:].values
Y = data_set.iloc[:,0].values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")
imputer.fit(X[:,:])
X[:,:] = imputer.transform(X[:,:])
print(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
#Random Forest
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 12, criterion = 'entropy', random_state=0)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

In [ ]:
def current_location():
    import requests
    r = requests.get('https://api.ipdata.co?api-key=31f7a5f46401e33ac72fca8d9242a023eea1ca8f4ff562f60b315a29').json()   
    location = [r['city'], r['country_name']]

    return location

location = current_location()
#location = ['New York', 'US']

def weather_conditions(location):    
    from pyowm.owm import OWM
    owm = OWM('444fb686d7df51b68b9504b0eb2778f7')
    mgr = owm.weather_manager()
    place = str(location[0]+','+location[1])    
    observation = mgr.weather_at_place(place)
    weather = observation.weather
    #observation = mgr.weather_at_place('New York','US')
    weather.status           
    weather.detailed_status
    #temp
    temp_k = weather.temperature()
    Celsius = temp_k['temp'] - 273.15
    Fahrenheit_t = Celsius*(9/5) + 32

    #wind chill
    temp_windchill = temp_k['feels_like'] - 273.15
    Fahrenheit_wc = Celsius*(9/5) + 32

    #humidity
    humidity = weather.humidity

    #pressure
    press = weather.pressure
    pressure = press['press']/51.715

    #visibility
    vis = weather.visibility_distance
    visibility = vis/1609

    #wind speed
    wind_speed = weather.wind()['speed']

    #parameters
    parameters = [Fahrenheit_t,Fahrenheit_wc,humidity,pressure,visibility,wind_speed]
    
    return parameters

In [ ]:
parameters = weather_conditions(location)
severity = classifier.predict([parameters])

In [ ]:
if(severity==1):
    print("Probability of accident is too low")
    print()
    print("Speed Limit must be below 80kmph")
elif(severity==2):
    print("Probability of accident is moderate")
    print()
    print("Speed Limit must be below 60kmph")

else:
    print("This location is prone to accidents⚠")
    print()
    print("Drive Safely!")
    print()
    print("Speed Limit must be below 45kmph")